In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlin-deeplearning-api:0.3.0")

import org.jetbrains.kotlinx.dl.api.core.Sequential
import org.jetbrains.kotlinx.dl.api.core.activation.Activations
import org.jetbrains.kotlinx.dl.api.core.initializer.GlorotNormal
import org.jetbrains.kotlinx.dl.api.core.initializer.Zeros
import org.jetbrains.kotlinx.dl.api.core.layer.core.Dense
import org.jetbrains.kotlinx.dl.api.core.layer.core.Input
import org.jetbrains.kotlinx.dl.api.core.loss.Losses
import org.jetbrains.kotlinx.dl.api.core.metric.Metrics
import org.jetbrains.kotlinx.dl.api.core.optimizer.Adam
import org.jetbrains.kotlinx.dl.api.core.summary.logSummary
import org.jetbrains.kotlinx.dl.dataset.OnHeapDataset
import kotlin.random.Random

In [2]:
val SEED = 12L
val TEST_BATCH_SIZE = 1000
val EPOCHS = 100
val TRAINING_BATCH_SIZE = 100

In [3]:
val model = Sequential.of(
    Input(4),
    Dense(1, Activations.Linear, kernelInitializer = GlorotNormal(), biasInitializer = Zeros(), name = "dense_2")
)

This example shows how to do regression from scratch, starting from generated dataset, using simple Dense-based [model] with 1 neuron.
It includes:
 - dataset creation
 - dataset splitting
 - model compilation
 - model training
 - model evaluation
 - model weights printing

In [4]:
val rnd = Random(SEED)

In [6]:
val data = Array(10000) { doubleArrayOf(0.0, 0.0, 0.0, 0.0, 0.0) }

for (i in data.indices) {
    data[i][1] = 2 * (rnd.nextDouble() - 0.5)
    data[i][2] = 2 * (rnd.nextDouble() - 0.5)
    data[i][3] = 2 * (rnd.nextDouble() - 0.5)
    data[i][4] = 2 * (rnd.nextDouble() - 0.5)
    data[i][0] = data[i][1] - 2 * data[i][2] + 1.5 * data[i][3] - 0.95 * data[i][4] + rnd.nextDouble(0.1)
    // 1 * x1 - 2 * x2 + 1.5 * x3  - 0.95 * x4 +- 0.1
}

data.shuffle()

In [7]:
fun extractX(): Array<FloatArray> {
    val init: (index: Int) -> FloatArray = { index ->
        floatArrayOf(
            data[index][1].toFloat(),
            data[index][2].toFloat(),
            data[index][3].toFloat(),
            data[index][4].toFloat()
        )
    }
    return Array(data.size, init = init)
}

In [8]:
fun extractY(): FloatArray {
    val labels = FloatArray(data.size) { 0.0f }
        for (i in labels.indices) {
        labels[i] = data[i][0].toFloat()
    }
    return labels
}

In [9]:
val dataset = OnHeapDataset.create(
    ::extractX,
    ::extractY
)

In [10]:
val (train, test) = dataset.split(0.9)

In [11]:
model.compile(
    optimizer = Adam(),
    loss = Losses.MSE,
    metric = Metrics.MAE
)

In [12]:
// temporary not support, https://github.com/JetBrains/KotlinDL/pull/251
model.logSummary()

In [13]:
model.fit(dataset = train, epochs = EPOCHS, batchSize = TRAINING_BATCH_SIZE)

org.jetbrains.kotlinx.dl.api.core.history.TrainingHistory@3bc735b3

In [14]:
repeat(100) { id ->
    val xReal = test.getX(id)
    val yReal = test.getY(id)
    val yPred = model.predictSoftly(xReal)

    println("xReal: ${arrayOf(xReal).contentDeepToString()}, yReal: $yReal, yPred: ${yPred[0]}")
}

xReal: [[-0.46381631, 0.22729328, -0.29862544, -0.21243858]], yReal: -1.1082815, yPred: -1.098101
xReal: [[0.5736741, 0.58325845, 0.7649467, -0.30839145]], yReal: 0.84929097, yPred: 0.93960464
xReal: [[-0.30228084, 0.3294686, -0.30368438, -0.6893691]], yReal: -0.71664935, yPred: -0.68782794
xReal: [[-0.1556095, -0.69278276, -0.59373754, -0.98509234]], yReal: 1.3380044, yPred: 1.2723669
xReal: [[0.3742811, 0.5068636, -0.1685736, 0.30070955]], yReal: -1.1212511, yPred: -1.0900835
xReal: [[-0.63312966, 0.7812809, -0.8235429, 0.41300496]], yReal: -3.7743382, yPred: -3.7140877
xReal: [[-0.2603415, -0.6877468, -0.18285686, 0.9571242]], yReal: -0.0061237975, yPred: -0.07070598
xReal: [[0.56727064, -0.12575187, 0.77977014, -0.40324146]], yReal: 2.3973203, yPred: 2.4099414
xReal: [[-0.82317954, -0.7354211, 0.258378, 0.047056086]], yReal: 1.0534166, yPred: 0.9833306
xReal: [[-0.51003575, -0.5526107, -0.5429335, 0.23280351]], yReal: -0.3731529, yPred: -0.43232405
xReal: [[0.22667238, -0.7503269, 

In [15]:
val mae = model.evaluate(dataset = test, batchSize = TEST_BATCH_SIZE).metrics[Metrics.MAE]
println("Weights: " + model.getLayer("dense_2").weights["dense_2_dense_kernel"].contentDeepToString())
println("Bias: " + model.getLayer("dense_2").weights["dense_2_dense_bias"].contentDeepToString())
println("MAE: $mae")

Weights: [[1.0004072], [-1.9244609], [1.4984584], [-0.9495823]]
Bias: [0.049071163]
MAE: 0.042641740292310715


In [16]:
repeat(100) { id ->
    val xReal = test.getX(id)
    val yReal = test.getY(id)

    val yPred = model.predictSoftly(xReal)

    println("xReal: ${xReal[0]}, yReal: $yReal, yPred: ${yPred[0]}")
}

xReal: -0.46381631, yReal: -1.1082815, yPred: -1.098101
xReal: 0.5736741, yReal: 0.84929097, yPred: 0.93960464
xReal: -0.30228084, yReal: -0.71664935, yPred: -0.68782794
xReal: -0.1556095, yReal: 1.3380044, yPred: 1.2723669
xReal: 0.3742811, yReal: -1.1212511, yPred: -1.0900835
xReal: -0.63312966, yReal: -3.7743382, yPred: -3.7140877
xReal: -0.2603415, yReal: -0.0061237975, yPred: -0.07070598
xReal: 0.56727064, yReal: 2.3973203, yPred: 2.4099414
xReal: -0.82317954, yReal: 1.0534166, yPred: 0.9833306
xReal: -0.51003575, yReal: -0.3731529, yPred: -0.43232405
xReal: 0.22667238, yReal: 3.5063016, yPred: 3.463846
xReal: -0.39176005, yReal: -1.0602486, yPred: -1.0150348
xReal: 0.44902572, yReal: -1.1682901, yPred: -1.1538981
xReal: 0.54311585, yReal: 0.7510916, yPred: 0.7059635
xReal: 0.46559408, yReal: 0.8290695, yPred: 0.805776
xReal: -0.72531503, yReal: -0.96634704, yPred: -0.91006887
xReal: -0.5366847, yReal: 0.39816996, yPred: 0.3346434
xReal: 0.41528165, yReal: 2.9658484, yPred: 2.8737

In [17]:
model.close()